# Notebook pour tester les traitements avant de les intégrer dans l'application du tableau de bord


In [ ]:
import  os
import pandas as pd
from datetime import datetime,timedelta
from google.cloud import bigquery

In [ ]:
issues=pd.read_csv('https://exploitidf.github.io/IssuesTunnels/_downloads/caee2520ef1721f8437118adc2284ddf/issuesFermetures.csv')#'CodeEx', 'PC', 'Tatouage', 'triCode', 'Fermeture'
pcFr=issues[[ 'PC', 'Fermeture']].drop_duplicates()
pcFr['Fermeture']=pcFr['Fermeture']+'$'
pcFr=pcFr.groupby('PC')['Fermeture'].sum().str[:-1]
pcFr=pcFr.str.split('$')
PCs=pcFr.index
pcFr=list(pcFr)
pcFrDict={PCs[i]:pcFr[i] for i in range(4) }

frTa=issues[[ 'Fermeture','Tatouage']].drop_duplicates()
frTa['Tatouage']=frTa['Tatouage']+'$'
frTa=frTa.groupby('Fermeture')['Tatouage'].sum().str[:-1]
frTa=frTa.str.split('$')
FERMs=frTa.index
frTa=list(frTa)
frTaDict={FERMs[i]:frTa[i] for i in range(len(frTa)) }
# frTaDict

In [ ]:
issues[issues['Tatouage']=='L10.372X']

,CodeEx,PC,Tatouage,triCode,Fermeture
5,IS106,PCO,L10.372X,A14,A14&NEU-Y


In [ ]:
pcFrDict.keys()

dict_keys(['PCE', 'PCN', 'PCO', 'PCS'])

In [ ]:


table =client.dataset("rapports_visites").table("LogDepot")
query = "SELECT * from `tunnels-dirif.rapports_visites.LogDepot` "
rows=client.query(query).result()
rowsTab=[list(row) for row in rows]
df=pd.DataFrame(rowsTab).drop_duplicates()
df.columns=['Tatouage', 'HoroDate', 'Agent', 'PC0', 'CM0', 'PC1', 'CM1', 'PC2', 'CM2', 'PC3', 'CM3', 'PC4', 'CM4', 'PC5', 'CM5', 'PC6', 'CM6',
                  'PC7', 'CM7', 'PC8', 'CM8', 'PC9', 'CM9', 'PC10', 'CM10', 'PC11', 'CM11', 'PC12', 'CM12', 'PC13', 'CM13', 'PC14', 'CM14']
df['Tatouage']=df['Tatouage'].str.strip()
df['dt']=  pd.to_datetime(df['HoroDate'],format='%Y-%m-%d %H:%M:%S')
df['date']= (df['dt']-pd.Timedelta(hours=12)).dt.strftime('%d-%m')
df=df.join(issues.set_index('Tatouage')['CodeEx'],on='Tatouage')


NameError: name 'client' is not defined

In [ ]:
client = bigquery.Client('tunnels-dirif')
query = "SELECT * from `tunnels-dirif.rapports_visites.VisiteIssuesFt` "
rows=client.query(query).result()
rowsTab=[list(row) for row in rows]
df=pd.DataFrame(rowsTab).drop_duplicates()
df.columns=['OTF','CodeEx','Tatouage','Agent','jour','Horodate','Fermeture','ordreVB','faitFerm','faitLocal' ]
df['Tatouage']=df['Tatouage'].str.strip()
df['Horodate']=df['Horodate'].str[:16]
df['dt']=  pd.to_datetime(df['Horodate'],format='%Y-%m-%d %H:%M')
df['date']= (df['dt']-pd.Timedelta(hours=12)).dt.strftime('%d-%m')

In [ ]:
controlesIsNi=pd.read_csv('https://raw.githubusercontent.com/ExploitIdF/TraitementRapportsVisitesBimestrielles/refs/heads/master/controlesIsNi.csv')#.iloc[:,:-1]
controlesIsNi.columns=['codePC', 'tPC', 'codeRC', 'tRC', 'Probab']


Index(['codePC', 'PC', 'codeRC', 'RC', 'Probab'], dtype='object')

In [ ]:
def litRC(tbNom):
    client = bigquery.Client('tunnels-dirif')
    query = "SELECT * from `tunnels-dirif.rapports_visites." +tbNom+"`"
    rows=client.query(query).result()
    rowsTab=[list(row) for row in rows]
    lstRC=pd.DataFrame(rowsTab).drop_duplicates()
    lstRC.columns=['ind','PC','RC','Com']
    lstRC['ind']=lstRC['ind'].astype(int)
    return lstRC

lstRCNi=litRC('lstRCNi')
lstRCIs=litRC('lstRCIs')


In [ ]:
def litVisite(tbNom):
    client = bigquery.Client('tunnels-dirif')
    query = "SELECT * from `tunnels-dirif.rapports_visites." +tbNom+"`"
    rows=client.query(query).result()
    rowsTab=[list(row) for row in rows]
    Visites=pd.DataFrame(rowsTab).drop_duplicates()
    Visites.columns=['OTF','CodeEx','Tatouage','Agent','jour','Horodate','Fermeture','ordreVB','faitFerm','faitLocal' ]
    return Visites

VisiteIssuesFt=litVisite('VisiteIssuesFt')
VisiteNichesFt=litVisite('VisiteNichesFt')


In [ ]:
VisiteIssuesFt[VisiteIssuesFt['faitLocal'].astype(int).astype(bool)]



,OTF,CodeEx,Tatouage,Agent,jour,Horodate,Fermeture,ordreVB,faitFerm,faitLocal
0,456605,IS106,L10.372X,Kim,36,2024-02-06 02:23:00,A14&NEU-Y,0,OK,1
1,456613,IS114,L10.381J,Kim,36,2024-02-06 00:29:00,A14&NEU-Y,0,OK,1
2,456616,IS117,L10.384M,Kim,36,2024-02-06 01:42:00,A14&NEU-Y,0,OK,1
3,456620,IS121,L10.388S,Kim,36,2024-02-05 23:11:00,A14&NEU-Y,0,OK,1
4,456683,IS185,L11.773P,Kim,36,2024-02-06 01:34:00,A14&NEU-Y,0,OK,1
...,...,...,...,...,...,...,...,...,...,...
1733,460840,IS334,L10.548H,Karen,337,2024-12-03 00:52:00,RUE-I,5,OK,1
1735,460807,IS474,L10.651F,Karen,338,2024-12-25 02:18:00,TAV,5,RP,1
1736,460656,IS172,L10.431T,Karen,339,2024-12-04 23:09:00,NAN-I,5,OK,1
1738,460814,IS313,L10.536T,Karen,339,2024-12-26 00:01:00,FRE&ANT-I,5,RP,1


In [ ]:
controleIS.columns
df[df['CodeEx']==iss].sort_values('dt')

,OT,CodeEx,Tatouage,Agent,jour,Horodate,dt,date
176,456601,IS102,L10.367S,Kun,38,2024-02-29 00:56,2024-02-29 00:56:00,28-02
177,456901,IS102,L10.367S,Kun,106,2024-04-16 02:53,2024-04-16 02:53:00,15-04
178,457201,IS102,L10.367S,Karl,149,2024-05-29 01:08,2024-05-29 01:08:00,28-05
179,457501,IS102,L10.367S,Kamel,224,2024-09-02 02:39,2024-09-02 02:39:00,01-09
180,457801,IS102,L10.367S,Karl,281,2024-10-28 23:03,2024-10-28 23:03:00,28-10
181,458101,IS102,L10.367S,Kun,323,2024-12-10 02:17,2024-12-10 02:17:00,09-12


In [ ]:
iss='IS102'
visIs=df[df['CodeEx']==iss].sort_values('dt')
lstRcIs=lstRC.join(visIs,on='ind',how='inner')
lstRcIs=lstRcIs.join(controleIS.set_index(['codePC','codeRC'])[['tPC','tRC']],on=['PC','RC'])
#lstRcIs=lstRcIs.join(controleIS.set_index('codeRC')['tRC'].drop_duplicates(),on='RC' ).sort_values(['Horodate','PC'])
lstRcIs=lstRcIs[['CodeEx','Horodate','PC','tRC','Com','Agent']].sort_values(['Horodate','PC'])
lstRcIs

,CodeEx,Horodate,PC,tRC,Com,Agent
22304,IS102,2024-02-29 00:56,P01,2_Propreté et vacuité OK,Commentaire Aléatoire3_Trois_Ècî,Kun
4569,IS102,2024-02-29 00:56,P02,2_Temps de fermeture en autonomie ≤ 10 secondes,Commentaire Aléatoire0_Très,Kun
3269,IS102,2024-02-29 00:56,P03,2_Surpression OK,Commentaire Aléatoire0_Très,Kun
18128,IS102,2024-02-29 00:56,P04,2_Propreté et vacuité sas OK,Commentaire Aléatoire4_Qûàtré,Kun
5747,IS102,2024-02-29 00:56,P05,2_Eclairage de secours OK,Commentaire Aléatoire2_Trop Û,Kun
...,...,...,...,...,...,...
915,IS102,2024-12-10 02:17,P11,2_Issue non équipée (?),Commentaire Aléatoire0_Très,Kun
17246,IS102,2024-12-10 02:17,P12,2_Issue non équipée (?),Commentaire Aléatoire4_Qûàtré,Kun
5752,IS102,2024-12-10 02:17,P13,2_TSE OK,Commentaire Aléatoire2_Trop Û,Kun
10541,IS102,2024-12-10 02:17,P14,2_Issue non équipée,Commentaire Aléatoire1_îàèù,Kun


In [ ]:
controleIS.set_index('codeRC')['tRC'].drop_duplicates()

codeRC
R0             0_Encombrement devant la porte coupe-feu
R1                          1_Autre anomalie à signaler
R2           1_Présence d'eau devant la porte coupe-feu
R3     1_Présence de détritus devant la porte coupe-feu
R4                             2_Propreté et vacuité OK
R0                            0_Porte coupe-feu bloquée
R1                      0_Porte coupe-feu dure à ouvrir
R2          0_Porte coupe-feu ne se refermant pas seule
R3                              0_Problème d'étanchéité
R4      0_Temps de fermeture en autonomie > 10 secondes
R6                         1_Porte coupe-feu endommagée
R7                                  2_Issue non équipée
R8                                 2_Porte coupe-feu OK
R9      2_Temps de fermeture en autonomie ≤ 10 secondes
R0    0_Surpression non activée à l'ouverture de la ...
R1    0_Surpression trop forte empêchant la seconde ...
R4                                     2_Surpression OK
R0    0_Encombrement critique du sas (pas